In [1]:
pip install numpy pandas yfinance matplotlib tensorflow scikit-learn


Note: you may need to restart the kernel to use updated packages.


In [6]:
import numpy as np
import pandas as pd
import yfinance as yf
import matplotlib.pyplot as plt
from sklearn.preprocessing import MinMaxScaler
from keras.models import Sequential
from keras.layers import LSTM, Dense, Dropout

# Function to fetch historical stock data for a specific stock symbol
def fetch_data(stock_symbol):
    # Fetch historical data from Yahoo Finance
    data = yf.download(stock_symbol + '.NS', start='2010-01-01', end='2023-12-01')  # NSE stocks use .NS suffix
    return data['Close'].values.reshape(-1, 1)

# Function to preprocess the data for LSTM model
def preprocess_data(prices):
    scaler = MinMaxScaler(feature_range=(0, 1))
    scaled_prices = scaler.fit_transform(prices)

    # Prepare training data
    X_train, y_train = [], []
    for i in range(60, len(scaled_prices)):
        X_train.append(scaled_prices[i-60:i])
        y_train.append(scaled_prices[i])
    
    return np.array(X_train), np.array(y_train), scaler

# Function to build and train the LSTM model
def build_and_train_model(X_train, y_train):
    model = Sequential()
    model.add(LSTM(units=50, return_sequences=True, input_shape=(X_train.shape[1], 1)))
    model.add(Dropout(0.2))
    model.add(LSTM(units=50))
    model.add(Dropout(0.2))
    model.add(Dense(units=1))  # Prediction of next price

    model.compile(optimizer='adam', loss='mean_squared_error')
    model.fit(X_train, y_train, epochs=10, batch_size=32)
    
    return model

# Function to make predictions
def predict_stock_price(model, scaler):
    prices = fetch_data('RELIANCE')  # Example: Reliance Industries Limited (NSE: RELIANCE)
    scaled_prices = scaler.transform(prices)

    # Prepare the last 60 days of data for prediction
    last_60_days = scaled_prices[-60:]
    X_test = np.array([last_60_days])

    predicted_price = model.predict(X_test)
    
    return scaler.inverse_transform(predicted_price)[0][0]

# Main function to execute the workflow
if __name__ == "__main__":
    stock_symbol = 'RELIANCE'  # Example: Reliance Industries Limited (NSE)
    
    # Fetch and preprocess data
    prices = fetch_data(stock_symbol)
    X_train, y_train, scaler = preprocess_data(prices)

    # Build and train the LSTM model
    model = build_and_train_model(X_train, y_train)

    # Make a prediction for the next day's price
    predicted_price = predict_stock_price(model, scaler)
    
    print(f'The predicted price for {stock_symbol} is: ₹{predicted_price:.2f}')
    

[*********************100%***********************]  1 of 1 completed

Epoch 1/10



C:\Users\Lenovo\anaconda3\Lib\site-packages\keras\src\layers\rnn\rnn.py:200: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


106/106 ━━━━━━━━━━━━━━━━━━━━ 5s 21ms/step - loss: 0.0304
Epoch 2/10
106/106 ━━━━━━━━━━━━━━━━━━━━ 2s 21ms/step - loss: 0.0023
Epoch 3/10
106/106 ━━━━━━━━━━━━━━━━━━━━ 2s 21ms/step - loss: 0.0018
Epoch 4/10
106/106 ━━━━━━━━━━━━━━━━━━━━ 2s 21ms/step - loss: 0.0020
Epoch 5/10
106/106 ━━━━━━━━━━━━━━━━━━━━ 2s 21ms/step - loss: 0.0017
Epoch 6/10
106/106 ━━━━━━━━━━━━━━━━━━━━ 2s 21ms/step - loss: 0.0018
Epoch 7/10
106/106 ━━━━━━━━━━━━━━━━━━━━ 2s 21ms/step - loss: 0.0017
Epoch 8/10
106/106 ━━━━━━━━━━━━━━━━━━━━ 2s 21ms/step - loss: 0.0016
Epoch 9/10
106/106 ━━━━━━━━━━━━━━━━━━━━ 2s 21ms/step - loss: 0.0015
Epoch 10/10
106/106 ━━━━━━━━━━━━━━━━━━━━ 2s 21ms/step - loss: 0.0015


[*********************100%***********************]  1 of 1 completed


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 227ms/step
The predicted price for RELIANCE is: ₹1163.78
